In [25]:
import folium
import json

# Load GeoJSON data
geojson_file_path = 'driver_1.geojson'
with open(geojson_file_path, 'r') as f:
  geojson_data = json.load(f)

# Assuming your GeoJSON is not empty, initialize the map with a default view
m = folium.Map()

# Add the GeoJSON layer to the map
folium.GeoJson(geojson_data, name="geojson").add_to(m)

# Loop through the GeoJSON features and add markers
for feature in geojson_data['features']:
  # Ensure the feature is a Point
  if feature['geometry']['type'] == 'Point':
    # Extract coordinates in [longitude, latitude] format
    lon, lat = feature['geometry']['coordinates']
    # Flip the coordinates for Folium's [latitude, longitude] format
    popup_message = feature['properties'].get('name', 'No Name')  # Customize as needed
    folium.Marker([lat, lon], popup=popup_message).add_to(m)

# Automatically adjust the map to show all markers
m.fit_bounds(m.get_bounds())

# Display the map in Jupyter Notebook
m

In [31]:
import openrouteservice
from openrouteservice import convert

from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv('ORS_API_KEY')

# Configure the client with your API key
client = openrouteservice.Client(key=API_KEY)

# Define coordinates (longitude, latitude) of start and end points
start_coord = (3.3700656, 6.5375897) 
end_coord = (3.433139, 6.456880200000001)  

# reverse coordinates to (latitude, longitude)
start_coord_rev = start_coord[::-1]
end_coord_rev = end_coord[::-1]

# Request route directions
routes = client.directions(coordinates=[start_coord, end_coord],
                           profile='cycling-regular',
                           format='geojson')

# Use Folium to visualize the route
import folium

# Create a map centered around the end point
map = folium.Map(location=end_coord_rev, zoom_start=13)

# Add the route to the map
folium.GeoJson(routes, name='route').add_to(map)

# Add markers for the start and end points
folium.Marker(start_coord_rev, popup='Start').add_to(map)
folium.Marker(end_coord_rev, popup='End').add_to(map)

# Display the map
map